# Descarga de imagenes

In [1]:
import ee
import geemap
from fastkml import kml
from fiona.drvsupport import supported_drivers
import fiona
import folium
import geojson
import geopandas as gpd
import os
#from geetools import batch
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import shape
from geojson import Polygon
import requests
import zipfile

## Geometria

In [2]:
#procesamiento de la geometria
supported_drivers['KML'] = 'rw'

geometriajson = gpd.read_file(os.path.join("datos","Aestudio84.kml"), driver="kml")
geometria_geojson = os.path.join("datos", "geometria.geojson")

geometriajson.to_file(geometria_geojson, driver="GeoJSON")

#geometria = [fila[:2] for fila in list(geometriajson.exterior[0].coords)]
#geometria

In [3]:
with open(geometria_geojson) as file:
    gj = geojson.load(file)

In [4]:
gj

{"crs": {"properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}, "type": "name"}, "features": [{"geometry": {"coordinates": [[[-75.000001, -36.000004, 0.0], [-66.999999, -36.0, 0.0], [-66.999999, -17.0, 0.0], [-75.000026, -16.999993, 0.0], [-75.000001, -36.000004, 0.0]]], "type": "Polygon"}, "properties": {"Description": "", "Name": "poligono"}, "type": "Feature"}], "type": "FeatureCollection"}

In [5]:
#codigo para convertir el kmz a kml
#en caso que no este presente
def conversion_kmz():
    path_kmz = os.path.join("datos", "Aestudio84.kmz")
    kml_doc = "doc.kml"
    with zipfile.ZipFile(path_kmz, 'r') as kmz:
        if kml_doc in kmz.namelist():
            kml_content = kmz.read(kml_doc).decode('utf-8')
    
            with open("Aestudio84.kml", 'w', encoding='utf-8') as kml_file:
                kml_file.write(kml_content)
    
            print("Archivo kml guardado")
        else:
            print("no se encontro el kml")

#conversion_kmz()

In [6]:
#Hardcodeo de datos
#fecha: cada dia del 2022
##observacion: probar con un dia
inicio = "2022-01-01"
fin = '2022-01-02'

In [7]:
prueba_gjson = gj["features"][0]["geometry"]["coordinates"][0]
prueba_gjson

[[-75.000001, -36.000004, 0.0],
 [-66.999999, -36.0, 0.0],
 [-66.999999, -17.0, 0.0],
 [-75.000026, -16.999993, 0.0],
 [-75.000001, -36.000004, 0.0]]

In [8]:
geometria_rounded = [[coord[0], coord[1]] for coord in prueba_gjson]
geometria_rounded

[[-75.000001, -36.000004],
 [-66.999999, -36.0],
 [-66.999999, -17.0],
 [-75.000026, -16.999993],
 [-75.000001, -36.000004]]

## carga

In [9]:
ee.Initialize(project="sergio-ramrez-agua-de-niebla")
preferencias = {"zoom": 22, "basemap": "Esri.WorldImagery",
                #"data_ctrl":False, "toolbar_ctrl":False, "draw_ctrl":False
}
#"center": [-27.3666, -70.3322]
map = geemap.Map(**preferencias)

region = ee.Geometry.Polygon(geometria_rounded)
#region = prueba_gjson
goes = ee.ImageCollection("NOAA/GOES/16/MCMIPF").filterBounds(region).filterDate(inicio, fin)
print("carga de imagenes hecha")


vis = {
    "bands": "CMI_C04",
    "min": 0.0,
    "max": 1.3,
    "gamma": 0.1
}

map.centerObject(region)
map.add_layer(region, {"color": 'FF0000'}, 'geodesic polygon')
print("primer layer añadido")
map.add_layer(goes.first().select("CMI_C04").clip(region), vis, "Zona Agua de Niebla")
print("primer layer añadido")
map

carga de imagenes hecha
primer layer añadido
primer layer añadido


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
out_dir = os.path.expanduser("~/Desktop/goes-images/")

geemap.download_ee_image_collection(goes.filterBounds(punto_aleatorio), out_dir=out_dir)

Total number of images: 144



Exception: Error downloading image collection: This image is unbounded, you need to specify a region; or a crs, crs_transform and shape.

In [11]:
punto_aleatorio = ee.Geometry.Point(-20.8263328, -70.148117).buffer(1000)
roi = ee.Geometry.Rectangle([-180, -90, 180, 90])

geemap.download_ee_image_collection(goes.filterBounds(punto_aleatorio) \
                         .select(["CMI_C04"]) \
                         , 
                         os.path.expanduser("~/Downloads/pruebas"), 
                         scale=10,crs=4326)

Total number of images: 144



Exception: Error downloading image collection: This image is unbounded, you need to specify a region; or a crs, crs_transform and shape.

In [13]:
out_dir = os.path.expanduser("~/Downloads")
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(loc)
    .filterDate("2008-01-01", "2020-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
geemap.download_ee_image_collection(collection, out_dir, scale=10)


Total number of images: 9



m_4609915_sw_14_060_20180902_20181213.tif: |                           | 0.00/1.49M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_060_20190626.tif: |                                    | 0.00/1.53M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_1_20090818.tif: |                                      | 0.00/1.72M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_1_20100629.tif: |                                      | 0.00/1.66M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_1_20120714.tif: |                                      | 0.00/1.66M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_1_20140901.tif: |                                      | 0.00/1.66M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_1_20150926.tif: |                                      | 0.00/1.66M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_h_20160704.tif: |                                      | 0.00/1.63M (raw) [  0.0%] in 00:00 (e…

m_4609915_sw_14_h_20170703.tif: |                                      | 0.00/1.49M (raw) [  0.0%] in 00:00 (e…

In [ ]:
def func_xmg(image):
  NUM_BANDS = 33
  bands = Array(NUM_BANDS)
  for i in range(1, 17, 1):
    bandName = 'CMI_C' + (100 + i + '').slice(-2)
    offset = ee.Number(image.get(bandName + '_offset'))
    scale =  ee.Number(image.get(bandName + '_scale'))
    bands[(i-1) * 2] = image.select(bandName).multiply(scale).add(offset)
    dqfName = 'DQF_C' + (100 + i + '').slice(-2)
    bands[(i-1) * 2 + 1] = image.select(dqfName)

  return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames())).toFloat()

scaleOffset = unsigned16.map(func_xmg)


In [2]:
from goes2go import GOES

G = GOES(satellite=16, product="ABI-L2-MCMIPF", domain='F')

In [16]:
 df = G.df(start='2022-07-04', end='2022-07-04')

DateParseError: Unknown datetime string format, unable to parse: 2022-07-04 Jul:00